# Intermediate Interdisciplinary Communication
### Exploration
## Sharing data insights with maps

In [1]:
# This code cell starts the necessary setup for Hour of CI lesson notebooks.
# First, it enables users to hide and unhide code by producing a 'Toggle raw code' button below.
# Second, it imports the hourofci package, which is necessary for lessons and interactive Jupyter Widgets.
# Third, it helps hide/control other aspects of Jupyter Notebooks to improve the user experience
# This is an initialization cell
# It is not displayed because the Slide Type is 'Skip'

from IPython.display import HTML, IFrame, Javascript, display
from ipywidgets import interactive
import ipywidgets as widgets
from ipywidgets import Layout

import getpass # This library allows us to get the username (User agent string)

# import package for hourofci project
import sys
sys.path.append('../../supplementary') # relative path (may change depending on the location of the lesson notebook)
import hourofci

# load javascript to initialize/hide cells, get user agent string, and hide output indicator
# hide code by introducing a toggle button "Toggle raw code"
HTML(''' 
    <script type="text/javascript" src=\"../../supplementary/js/custom.js\"></script>
    
    <style>
        .output_prompt{opacity:0;}
    </style>
    
    <input id="toggle_code" type="button" value="Toggle raw code">
''')

A great deal of the scientific communication we see today is accompanied by maps as they are excellent ways to make data immediately comprehensible. 

It's now very easy for anyone to make maps, BUT it is equally easy to make bad and misleading maps. 

We don't have time to go into this thoroughly now - there's a whole discipline called <b><i>Cartography</i></b> that addresses the Art and Science of Mapmaking. 

But to give you a quick taste, let's look at how we can easily change the message delivered on a map with a simple variable change in our code. 

Click the X at the top left now to drop into a notebook and follow along...

## Let's start with a simple Choropleth map.
A choropleth map divides the geography of interest into a set of adjoining zones (such as the 48 contiguous states) which are colored according to values assigned to each zone. If you completed the Introduction to Geospatial Data lesson you made such a map from covid rates data. Let's use that code again here, but play around with visualization aspects.

We start as always by importing the necessary packages. 

In [ ]:
#import the necessary packages
import geopandas
import json
import pandas

Then we import the polygons representing the states and trim the set to only the 48 contiguous ones. 

In [ ]:
#import the polygons representing the states and trim to only the 48 contiguous states
states = geopandas.read_file("http://www2.census.gov/geo/tiger/GENZ2014/shp/cb_2014_us_state_20m.zip")

#crop off Alaska and Hawaii
from shapely.geometry import Polygon
mask = Polygon([(-140, 20), (-140, 50), (-60, 50), (-60, 20), (-140, 20)])
states = geopandas.clip(states, mask)
states.plot()

OK now we have the geography, let's get the covid data from Johns Hopkins University's remarkable repository (https://coronavirus.jhu.edu/). In the code below we get the data for October 2, 2020. You can come back later and try other dates. 

We will merge the two datasets using the fields containing the state name. Unfortunately the two datasets use different names for that field, so we have to do a field rename before merging. Here's the code: 

In [ ]:
#get the data
cases = pandas.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_daily_reports_us/10-02-2020.csv')

#rename one field so that we can merge the polygons with the data
cases = cases.rename(columns={'Province_State': 'NAME'})

#merge the data with the polygons
states = states.merge(cases, on='NAME')
states.head()

Scroll to the right and you'll see the covid rates data appended to the states data. 

OK, now we can play making maps using the following code:

<code>states.plot(column='variable', cmap='value', scheme='value')</code>

In this single line of code are two parameters that will change the appearance of your maps completely: <b>cmap</b> and <b>scheme</b>.

### cmap

The colormap ('cmap') parameter  sets the range of colors used. There are LOTS of colormaps!

But quickly just for a look, here are four very different ones:
- 'Greens' is a graduated range of greens from light to dark. One like this is best for this data. You can quickly tell which states have high values and which are the lowest, even without looking at the legend. 

In [ ]:
states.plot(column='Incident_Rate', cmap='Greens', scheme='quantiles', legend=True, figsize=(10, 10))

Now let's take a look at a couple of colormaps that are inappropriate for this data. Can you tell which are the highest values without looking at the legend?

- 'Set1' is non-ordered set of colors. This is absolutely the wrong one to use.

In [ ]:
states.plot(column='Incident_Rate', cmap='Set1', scheme='quantiles', legend=True, figsize=(10, 10))

Other color maps are for diverging data that range from negative numbers to positive numbers. 
- 'Spectral' is one of these. Since these data are rates of illness, values range from close to 0 to a high number. This is not appropriate for this data. 

BUT you can use it incorrectly if you don't know any better! See the code below where we do just that. If you were trying to understand where the highest rates are, could you quickly tell without using the legend?

In [ ]:
states.plot(column='Incident_Rate', cmap='Spectral', scheme='quantiles', legend=True, figsize=(10, 10))

Now scroll back through these maps we just made. Which one is the one you should put in a scientific report for the general public about covid incidence rates?

Remember that your choice of colors is critical in telling a meaningful story with maps. 

If you want to see the many colormaps, check this link in the relevant Python manual:https://matplotlib.org/stable/tutorials/colors/colormaps.html. 

### Scheme

Even more important to making a map that is easily interpreted, is the Scheme parameter. This is how you classify the data, i.e. how you divide up the range of data values. It is very easy to make misleading maps by choosing the wrong one of these.

It's easiest to understand this if we start with a histogram of the data we're mapping.

In [ ]:
states.hist(column='Incident_Rate')

Notice that the data is not equally spread across the range of values. There are only 2 states below about 600 and 3 above 3200. The most states have values between about 1200 and 3000. 

If we want to map the data with 5 classes like we did earlier, then we have to put 4 break points along the data range. How to choose where these go?

There are many choices for the scheme parameter, including ‘equal_interval’, ‘fisher_jenks’, ‘maximum_breaks’, ‘natural_breaks’, ‘quantiles’, ‘percentiles’, and ‘std_mean’. 

Let's start with 'quantiles'. This divides the data into equal sized bins, each containing 20% of the data values. Thus the lowest quantile has the 9 or 10 states with the lowest values (20% of 48 = 9.6). The top bin has the 9 or 10 states with the highest values.

In [ ]:
states.plot(column='Incident_Rate', cmap='Greens', scheme='quantiles', legend=True, figsize=(10, 10))

That's a useful way to look at this data.

Another one that works for this data is 'equal_interval'. Here the full data range is broken up into equal intervals, irregardless of how many states are in each interval. 

Let's see how this changes our map:

In [ ]:
states.plot(column='Incident_Rate', cmap='Greens', scheme='equal_interval', legend=True, figsize=(10, 10))

This one helps emphasize the outliers. Where are the states with very low rates and which have very high rates? It tells a slightly different story.

Our next scheme, 'maximum_breaks' is very different and depends on a mathematical calculation. The idea is that the class breaks fall where the data itself is most spread out. 

In [ ]:
states.plot(column='Incident_Rate', cmap='Greens', scheme='maximum_breaks', legend=True, figsize=(10, 10))

This one helps you see where the data clusters are. Same data, very different maps!

That's it for our exploration. If you've still got some time to play, go back and get a different data set, try different colormaps and different classification schemes. See if you can get the same data to apparently tell two opposite stories. 

# Congratulations!


**You have finished an Hour of CI!**


But, before you go ... 

1. Please fill out a very brief questionnaire to provide feedback and help us improve the Hour of CI lessons. It is fast and your feedback is very important to let us know what you learned and how we can improve the lessons in the future.
2. If you would like a certificate, then please type your name below and click "Create Certificate" and you will be presented with a PDF certificate.

<font size="+1"><a style="background-color:blue;color:white;padding:12px;margin:10px;font-weight:bold;" href="https://forms.gle/JUUBm76rLB8iYppN7">Take the questionnaire and provide feedback</a></font>


In [3]:

# This code cell loads the Interact Textbox that will ask users for their name
# Once they click "Create Certificate" then it will add their name to the certificate template
# And present them a PDF certificate
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw

from ipywidgets import interact

def make_cert(learner_name, lesson_name):
    cert_filename = 'hourofci_certificate.pdf'

    img = Image.open("../../supplementary/hci-certificate-template.jpg")
    draw = ImageDraw.Draw(img)

    cert_font   = ImageFont.truetype('../../supplementary/cruft.ttf', 150)
    cert_fontsm = ImageFont.truetype('../../supplementary/cruft.ttf', 80)
    
    _,_,w,h = cert_font.getbbox(learner_name)  
    draw.text( xy = (1650-w/2,1100-h/2), text = learner_name, fill=(0,0,0),font=cert_font)
    
    _,_,w,h = cert_fontsm.getbbox(lesson_name)
    draw.text( xy = (1650-w/2,1100-h/2 + 750), text = lesson_name, fill=(0,0,0),font=cert_fontsm)
    
    img.save(cert_filename, "PDF", resolution=100.0)   
    return cert_filename


interact_cert=interact.options(manual=True, manual_name="Create Certificate")

@interact_cert(name="Your Name")
def f(name):
    print("Congratulations",name)
    filename = make_cert(name, 'Intermediate Interdisciplinary Communication')
    print("Download your certificate by clicking the link below.")
    
    
    

interactive(children=(Text(value='Your Name', description='name'), Button(description='Create Certificate', st…

<font size="+1"><a style="background-color:blue;color:white;padding:12px;margin:10px;font-weight:bold;" href="hourofci_certificate.pdf?download=1" download="hourofci_certificate.pdf">Download your certificate</a></font>

